In [2]:
import mne
import numpy as np
import tensorflow.keras
import os

2022-07-05 00:59:01.699358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guisoares/catkin_ws/devel/lib:/opt/ros/noetic/lib:/usr/lib/x86_64-linux-gnu/:/home/guisoares/catkin_ws/install/lib/mavlink_sitl_gazebo/plugins
2022-07-05 00:59:01.699386: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def read_edf_to_raw(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.set_eeg_reference()
    raw.filter(l_freq=0.5, h_freq=45)
    return raw

In [4]:
seizures_dict = {"chb01_03": [[2996, 3036]],
                "chb01_04": [[1467, 1494]],
                "chb01_15": [[1732, 1772]],
                "chb01_16": [[1015, 1066]],
                "chb01_18": [[1720, 1810]],
                "chb01_21": [[327, 420]],
                "chb01_26": [[1862, 1963]],
                
                "chb02_16": [[130, 212]],

                "chb05_06": [[417, 532]], 
                "chb05_13": [[1086, 1196]],
                "chb05_16": [[2317, 2413]], 
                "chb05_17": [[2451, 2571]],
                "chb05_22": [[2348, 2465]],
                
                "chb08_02": [[2670, 2841]], 
                "chb08_05": [[2856, 3046]],
                "chb08_11": [[2988, 3211]], 
                "chb08_13": [[2417, 2577]],
                "chb08_21": [[2083, 2347]]}

In [ ]:
################ INPUT YOUR DATASET PATH HERE
DATASET_PATH = os.path.join(os.getcwd(), 'data') #ONLY FOR TESTING

################ CHOOSE THE FILES THAT YOU WANT HERE
selected_files = {'chb01': ['01','02','03','04','15','16','18','21','26']}
files = [os.path.join(DATASET_PATH, folder, f"{folder}_{fn}.edf") for folder, fn_list in selected_files.items() for fn in fn_list]

for file_path in files:
    print(file_path)


# parameters for epochs generation
epoch_time = 100
overlap = 0

# Divide into epochs
signals = []
labels = []
for file_path in files:
    # get filename: chbxx_xx.edf
    filename = os.path.split(file_path)[1]
    # remove .edf staying only chbxx_xx
    filename = os.path.splitext(filename)[0]
    
    raw = read_edf_to_raw(file_path)
    
    # epochs = mne.make_fixed_length_epochs(raw, duration=epoch_time, preload=True, overlap=overlap)

    curr_time = 0
    while curr_time <= max(raw.times) - epoch_time:

        # calculate window and get data to epoch array
        start_time = curr_time 
        if start_time < 0.:
            start_time = 0.
        end_time = curr_time + epoch_time

        start, stop = raw.time_as_index([start_time, end_time])
        epoch = raw[:, start:stop][0]
        signals.append(epoch)

        # seizure flag for y
        aux = []
        if filename in seizures_dict:  # if file has seizure
            for seizure in seizures_dict[filename]:
                if start_time > seizure[0] and start_time < seizure[1]:
                    aux.append(1)
                if start_time + epoch_time > seizure[0] and start_time + epoch_time < seizure[1]:
                    aux.append(1)
                if start_time < seizure[0] and end_time > seizure[0]:
                    aux.append(1)
                if start_time < seizure[0] and end_time > seizure[1]:
                    aux.append(1)
                else:
                    aux.append(0)
        else:    
            aux.append(0)
        
        # if the current time is inside at least one seizure interval -> 1; otherwise -> 0
        if 1 in aux:
            labels.append([1])
        else:
            labels.append([0])

        # calculate next current time
        curr_time = curr_time + epoch_time - overlap
        print("Section ", str(len(signals)), "; start: ", start, " ; stop: ", stop)
        
signals = np.array(signals)
labels = np.array(labels)


In [6]:
signals.shape

(303, 23, 25600)

In [8]:
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 292, 1: 11}


In [9]:
np.save('signals/chb01/signals.npy',signals)
np.save('signals/chb01/labels.npy',labels)